# FWI Data Exploration

Exploratory analysis of the Fire Weather Index dataset for super-resolution research.

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from data_processing import FWIDataProcessor

%matplotlib inline

In [ ]:
# Initialize data processor
processor = FWIDataProcessor('../configs/params.yaml')

# Load dataset
dataset = processor.load_unified_dataset()
print(f"Dataset dimensions: {dataset.dims}")
print(f"Variables: {list(dataset.data_vars)}")

In [ ]:
# Plot FWI statistics
if 'fwi' in dataset.data_vars:
    fwi_data = dataset['fwi']
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Sample day
    sample_day = fwi_data.isel(time=100)
    im1 = axes[0,0].imshow(sample_day, cmap='RdYlBu_r', vmin=0, vmax=50)
    axes[0,0].set_title('FWI - Sample Day')
    plt.colorbar(im1, ax=axes[0,0])
    
    # Time series at a point
    point_ts = fwi_data.isel(latitude=300, longitude=180)
    axes[0,1].plot(point_ts.time, point_ts)
    axes[0,1].set_title('FWI Time Series - Sample Location')
    axes[0,1].set_xlabel('Date')
    axes[0,1].set_ylabel('FWI')
    
    # Distribution
    fwi_values = fwi_data.values.flatten()
    fwi_values = fwi_values[~np.isnan(fwi_values)]
    axes[1,0].hist(fwi_values, bins=50, alpha=0.7)
    axes[1,0].set_title('FWI Distribution')
    axes[1,0].set_xlabel('FWI Value')
    axes[1,0].set_ylabel('Frequency')
    
    # Statistics
    stats_text = f"""
    Statistics:
    Mean: {np.nanmean(fwi_values):.2f}
    Std: {np.nanstd(fwi_values):.2f}
    Min: {np.nanmin(fwi_values):.2f}
    Max: {np.nanmax(fwi_values):.2f}
    Median: {np.nanmedian(fwi_values):.2f}
    
    Dataset Info:
    Shape: {fwi_data.shape}
    Non-NaN: {np.sum(~np.isnan(fwi_values)):,}
    NaN: {np.sum(np.isnan(fwi_data.values)):,}
    """
    
    axes[1,1].text(0.1, 0.5, stats_text, transform=axes[1,1].transAxes, 
                   fontsize=10, verticalalignment='center')
    axes[1,1].set_title('Dataset Statistics')
    axes[1,1].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Validate fire detection
fire_detected = processor.validate_fire_detection(dataset)
print(f"Pedrógão Grande fire detection: {'✅' if fire_detected else '❌'}")